<a href="https://colab.research.google.com/github/shatatoy/tatoy/blob/main/SBERT_large_mt_NLU_ru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q transformers

     |████████████████████████████████| 2.6 MB 33.8 MB/s 
     |████████████████████████████████| 636 kB 60.0 MB/s 
     |████████████████████████████████| 3.3 MB 58.8 MB/s 
     |████████████████████████████████| 895 kB 59.6 MB/s 


In [4]:
from transformers import AutoTokenizer, AutoModel
  
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_mt_nlu_ru")
model = AutoModel.from_pretrained("sberbank-ai/sbert_large_mt_nlu_ru")

In [5]:
import torch

In [6]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [7]:

#Sentences we want sentence embeddings for
sentences = ['Привет! Как твои дела?',
             'А правда, что 42 твое любимое число?']
#Load AutoModel from huggingface model repository
# tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_mt_nlu_ru")
# model = AutoModel.from_pretrained("sberbank-ai/sbert_large_mt_nlu_ru")
#Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=24, return_tensors='pt')

In [8]:
#Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

In [9]:
#Perform pooling. In this case, mean pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

In [10]:
sentence_embeddings

tensor([[ 0.1246, -0.3668, -1.2310,  ...,  0.7144, -1.3901, -0.2763],
        [ 0.1691, -0.3796, -0.0374,  ...,  0.2068, -0.6518,  0.9915]])

In [11]:
sentence_embeddings.shape

torch.Size([2, 1024])